In [2]:
import gc
import glob
import json
import os
import math
from collections import namedtuple
from os.path import join as pjoin
import codecs
from multiprocessing import Pool
import multiprocessing.pool
import numpy as np
from collections import defaultdict
from itertools import groupby

import sentencepiece
from gensim import corpora, models, similarities
# from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize

In [3]:
json_para = json.load(open(r"E:\graphsum\data\analyze_sim_graph_weights\MultiNews.30.train.11.json", encoding="utf-8"))
json_sent = json.load(open(r"E:\graphsum\data\analyze_sim_graph_weights\sent_MultiNews.30.train.11.json", encoding="utf-8"))

json_paddle = json.load(open(r"E:\graphsum\data\MultiNews_data_tfidf_30_paddle\train\MultiNews.30.train.11.json", encoding="utf-8"))

In [4]:
spm = sentencepiece.SentencePieceProcessor()
spm.load(r"E:\graphsum\data\spm9998_3.model")

True

In [279]:
save = 0
total_sent = 0
for ex in json_sent:
    src_filtered_short = src_filtered_short = [[spm.decode_ids(
                [token]) for token in sent] for sent in ex["src"]]
    unpruned_length = len(src_filtered_short)
    
    pruned = prune_input(src_filtered_short)
    
    pruned_length = unpruned_length - len(pruned)
    
    total_sent += unpruned_length
    
    save += pruned_length
    

The corpus_tfidf[i] is None: 1
[(1, 1)]
The corpus_tfidf[i] is None: 13
[(0, 1)]
The corpus_tfidf[i] is None: 49
[(0, 1)]
The corpus_tfidf[i] is None: 21
[(1, 1)]
The corpus_tfidf[i] is None: 38
[(1, 1)]
The corpus_tfidf[i] is None: 27
[(3, 1)]
The corpus_tfidf[i] is None: 47
[(1, 1)]
The corpus_tfidf[i] is None: 79
[(1, 1)]
The corpus_tfidf[i] is None: 22
[(0, 1)]
The corpus_tfidf[i] is None: 39
[(2, 1)]
The corpus_tfidf[i] is None: 49
[(0, 1)]
The corpus_tfidf[i] is None: 9
[(2, 1)]
The corpus_tfidf[i] is None: 29
[(2, 1)]
The corpus_tfidf[i] is None: 31
[(2, 1)]
The corpus_tfidf[i] is None: 1
[(1, 1)]
The corpus_tfidf[i] is None: 40
[(1, 1)]
The corpus_tfidf[i] is None: 77
[(1, 1)]
The corpus_tfidf[i] is None: 37
[(1, 1)]
The corpus_tfidf[i] is None: 21
[(1, 1)]
The corpus_tfidf[i] is None: 25
[(1, 1)]
The corpus_tfidf[i] is None: 33
[(1, 1)]
The corpus_tfidf[i] is None: 7
[(0, 1)]
The corpus_tfidf[i] is None: 19
[(0, 1)]
The corpus_tfidf[i] is None: 20
[(0, 1)]
The corpus_tfidf[i] 

The corpus_tfidf[i] is None: 45
[(1, 1)]
The corpus_tfidf[i] is None: 62
[(1, 1)]
The corpus_tfidf[i] is None: 51
[(0, 1)]
The corpus_tfidf[i] is None: 4
[(0, 1)]
The corpus_tfidf[i] is None: 25
[(2, 1)]
The corpus_tfidf[i] is None: 28
[(2, 1)]
The corpus_tfidf[i] is None: 30
[(2, 1)]
The corpus_tfidf[i] is None: 20
[(2, 1)]
The corpus_tfidf[i] is None: 42
[(2, 1)]
The corpus_tfidf[i] is None: 23
[(0, 1)]
The corpus_tfidf[i] is None: 1
[(2, 1)]
The corpus_tfidf[i] is None: 37
[(2, 1)]
The corpus_tfidf[i] is None: 51
[(2, 1)]
The corpus_tfidf[i] is None: 58
[(2, 1)]
The corpus_tfidf[i] is None: 19
[(1, 1)]
The corpus_tfidf[i] is None: 30
[(1, 1)]
The corpus_tfidf[i] is None: 33
[(1, 1)]
The corpus_tfidf[i] is None: 34
[(1, 1)]
The corpus_tfidf[i] is None: 35
[(1, 1)]
The corpus_tfidf[i] is None: 37
[(1, 1)]
The corpus_tfidf[i] is None: 80
[(1, 1)]
The corpus_tfidf[i] is None: 11
[(2, 1)]
The corpus_tfidf[i] is None: 25
[(2, 2)]
The corpus_tfidf[i] is None: 24
[(2, 1)]
The corpus_tfidf[i

In [280]:
print(f"total_sent: {total_sent}, save: {save}, percentage: {save/total_sent}")

total_sent: 44441, save: 404, percentage: 0.009090704529601043


In [197]:
def prune_input(corpus):
    raw_corpus = [' '.join(para) for para in corpus]
       
    # create English stop words list
    stoplist = set(stopwords.words('english'))
    
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()

    # Lowercase each document, split it by white space and filter out stopwords
    texts = [[word for word in para.lower().split() if word not in stoplist]
             for para in raw_corpus]
    # Create a set of frequent words
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1

    # stem each word
    processed_corpus = [[p_stemmer.stem(
        token) for token in text if frequency[token] > 1] for text in texts]
   
    dictionary = corpora.Dictionary(processed_corpus)
    bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

    # train the model
    tfidf = models.TfidfModel(bow_corpus)
    # transform the "system minors" string
    corpus_tfidf = tfidf[bow_corpus]

    to_remove = []
    for i, cor in enumerate(corpus_tfidf):
        if len(cor) == 0:
            print("The corpus_tfidf[i] is None: %d" % i)
            print(bow_corpus[i])
            to_remove.append(i)
            # exit(1)

    
    valid_indexes = [x for x in range(len(corpus_tfidf)) if x not in to_remove]
    
    return [sent for i,sent in enumerate(corpus) if i in valid_indexes]
    

In [203]:
def construct_tfidf_sim_graph_by_gensim(corpus, sim_threshold):
    """Constuct TFIDF similarity graph by Gensim package"""

    sim_graph = []
    
    ### Trenne Buchstabend von einander
    raw_corpus = [' '.join(para) for para in corpus]
    
    # create English stop words list
    stoplist = set(stopwords.words('english'))
    
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()

    # Lowercase each document, split it by white space and filter out stopwords
    texts = [[word for word in para.lower().split() if word not in stoplist]
             for para in raw_corpus]
    # Create a set of frequent words
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1

    # stem each word
    processed_corpus = [[p_stemmer.stem(
        token) for token in text if frequency[token] > 1] for text in texts]
   
    dictionary = corpora.Dictionary(processed_corpus)
    bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

    # train the model
    tfidf = models.TfidfModel(bow_corpus)
    # transform the "system minors" string
    corpus_tfidf = tfidf[bow_corpus]
    
    index = similarities.SparseMatrixSimilarity(
        corpus_tfidf, num_features=len(dictionary))

    total = 0.
    count_large = 0.
    for i in range(len(corpus_tfidf)):
        sim = index[corpus_tfidf[i]]
        assert len(sim) == len(corpus_tfidf), "the tfidf sim is not correct!"
        sim_graph.append(sim.tolist())

        for s in sim:
            total += 1
            if s > sim_threshold:
                count_large += 1

    print("sim_graph[0]: %s" % str(sim_graph[0]))

    return sim_graph, count_large, total

In [262]:
src_filtered_short = [[spm.decode_ids(
                [token]) for token in sent] for sent in json_sent[3]["src"]]

In [263]:
pruned_input = prune_input(src_filtered_short)

In [271]:
sim_graph, count_large, total = construct_tfidf_sim_graph_by_gensim(pruned_input,0.05)

sim_graph[0]: [1.0, 0.0, 0.13951730728149414, 0.0, 0.0, 0.0, 0.310952752828598, 0.0, 0.0, 0.0, 0.0, 0.6070091724395752, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14977853000164032, 0.0, 0.0, 0.0, 0.35957664251327515, 0.0, 0.0]


In [272]:
np.mean(sim_graph)

0.13976133677109223

In [273]:
para_src_tokens = [[spm.decode_ids(
                [token]) for token in sent] for sent in json_para[3]["src"]]

In [274]:
p_sim_graph, count_large, total = construct_tfidf_sim_graph_by_gensim(para_src_tokens,0.05)

sim_graph[0]: [1.0, 0.0, 0.0984400287270546, 0.0, 0.2873874008655548, 0.0, 0.0, 0.0, 0.5833762288093567, 0.0, 0.0, 0.0, 0.10950969904661179, 0.0, 0.36199307441711426, 0.0, 0.0]


In [275]:
np.mean(paddle_sim_graph)

0.09768263687198568